# Two-compartment uptake model (2CUM)

In [ ]:
# import statements
import os
import numpy as np
from matplotlib import pyplot as plt
import csv
import pandas as pd
import seaborn as sns
from plotting_results_nb import plot_bland_altman, bland_altman_statistics, make_catplot
import json
from pathlib import Path

## Test data

Data summary: simulated two-compartment uptake model data

Source: Concentration-time data (n = 27) generated by M. Thrippleton using Matlab code at https://github.com/mjt320/DCE-functions

Detailed info:
- Temporal resolution: 0.5 s
- Acquisition time: 300 s
- AIF: Parker function, starting at t=10s
- Noise: SD = 0.001 mM
- Arterial delay: none or 5s
Since it is challenging to fit all parameters across a wide area of parameter space, data is generated with high SNR.
 
Reference values: Reference values are the parameters used to generate the data. All combinations of $v_p$ (0.02 to 0.1), $f_p$ (5 to 40 100ml/ml/min) and PS (0.0001 to 0.25 per min) are included.
A delayed version of the test data was created by shifting the time curves with 5s. This data is labeled as 'delayed' and only used with the models that allow the fitting of a delay.

Citation: Code used in Manning et al., Magnetic Resonance in Medicine, 2021 https://doi.org/10.1002/mrm.28833 and Matlab code: https://github.com/mjt320/DCE-functions

Tolerances
- $v_p$: a_tol=0.025, r_tol=0, start=0.01, bounds=(0,1)
- $f_p$: a_tol=5, r_tol=0.1, start=20, bounds=(0,200), units ml/100ml/min
- PS: a_tol=0.005, r_tol=0.1, start=0.6, bounds=(0,5), units [/min]
- delay: a_tol=0.5, r_tol=0, start=0, bounds=(-10,10), units s




### Visualize test data
To get an impression of the test data, the concentration time curves of the test data are plotted below.

In [ ]:
#plot test data
filename = ('../test/DCEmodels/data/2cum_sd_0.0025_delay_0.csv')
# read from CSV to pandas
df1 = pd.read_csv(filename)

no_voxels = len(df1.label)

fig, ax = plt.subplots(1, 1, sharex='col', sharey='row', figsize=(6,6))
for currentvoxel in range(no_voxels):
    labelname = 'case_' + str(currentvoxel+1)
    testdata = df1[(df1['label']==labelname)]
    t = testdata['t'].to_numpy()
    t = np.fromstring(t[0], dtype=float, sep=' ')
    c = testdata['C_t'].to_numpy()
    c = np.fromstring(c[0], dtype=float, sep=' ')
    ax.plot(t, c, label=labelname)
    
ax.set_ylabel('C (mM)', fontsize=14)
ax.set_xlabel('time (ms)', fontsize=14)
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0,fontsize=9);   
plt.xticks(fontsize=14)
plt.yticks(fontsize=14);

    

## Import data

In [ ]:
# Load the meta data
meta = json.load(open("../test/results-meta.json"))

In [ ]:
# Loop over each entry and collect the dataframe
df = []
for entry in meta:
    if (entry['category'] == 'DCEmodels') & (entry['method'] == '2CUM') :
        fpath, fname, category, method, author = entry.values()
        df_entry = pd.read_csv(Path(fpath, fname)).assign(author=author)
        df.append(df_entry)
    
# Concat all entries
df = pd.concat(df)

In [ ]:
author_list = df.author.unique()
no_authors = len(author_list)

# split delayed and non-delayed data
df['delay'] = df['label'].str.contains('_delayed')

# calculate error between measured and reference values
df['error_ps'] = df['ps_meas'] - df['ps_ref']
df['error_vp'] = df['vp_meas'] - df['vp_ref']
df['error_fp'] = df['fp_meas'] - df['fp_ref']

# tolerances
tolerances = { 'ps': {'atol' : 0.005, 'rtol': 0.1 }, 'vp': {'atol':0.025, 'rtol':0}, 've': {'atol':0.05, 'rtol':0}, 'fp': {'atol':5, 'rtol':0.1}}

## Results

### Non-delayed data

Some models allow the fit of a delay. For the tests with non-delayed data, the delay was fixed to 0.

If the tolerance lines are not shown, it means that they are outside the limits of the y-axis.

In [ ]:
df.head(n=27)[['label','ps_ref','vp_ref', 'fp_ref']]

In [ ]:
# set-up styling for seaborn plots
sns.set(font_scale=1.5)
#sns.set_style("whitegrid")
sns.set_style("ticks")
plotopts = {"hue":"author",
            "dodge":True,
            "s":100,
            "height":4,
            "aspect":3
           }

In [ ]:
make_catplot(x='label', y="error_ps", data=df[~df['delay']], 
             ylabel="$\Delta$ PS ($min^{-1}$)", **plotopts)

Bias results estimated PS values combined for all voxels

In [ ]:
make_catplot(x='label', y="error_vp", data=df[~df['delay']], 
             ylabel="$v_{p,ref}$ (-)", **plotopts)

In [ ]:
make_catplot(x='label', y="error_fp", data=df[~df['delay']], 
             ylabel="$f_{p,ref}$ (-)", **plotopts)

Bias results estimated PS values combined for all voxels

In [ ]:
resultsBA = bland_altman_statistics(data=df[~df['delay']],par='error_ps',grouptag='author')
print(resultsBA)

Bias results estimated $v_p$ values combined for all voxels

In [ ]:
resultsBA = bland_altman_statistics(data=df[~df['delay']],par='error_vp',grouptag='author')
print(resultsBA)

Bias results estimated $f_p$ values combined for all voxels

In [ ]:
resultsBA = bland_altman_statistics(data=df[~df['delay']],par='error_fp',grouptag='author')
print(resultsBA)

### Delayed results

Some contributions allowed the fitting of a delay. For those, additional tests with a delay were performed.

In [ ]:
make_catplot(x='label', y="error_ps", data=df[df['delay']], 
             ylabel="$\Delta$ PS ($min^{-1}$)", **plotopts)

Bias results estimated PS values combined for all voxels

In [ ]:
make_catplot(x='label', y="error_vp", data=df[df['delay']], 
             ylabel="$v_{p,ref}$ (-)", **plotopts)

In [ ]:
make_catplot(x='label', y="error_fp", data=df[df['delay']], 
             ylabel="$f_{p,ref}$ (-)", **plotopts)

## Notes
Additional notes/remarks

### References